# 计算区域排水能力

In [1]:
import os
import arcpy
from arcpy.sa import *

In [2]:
# 设置环境参数
arcpy.env.workspace =r"C:\Users\Runker\Desktop\DEM_test\drainwater.gdb"
arcpy.env.overwriteOutput = True
# 设置单进程模式
arcpy.env.parallelProcessingFactor = "1"

In [4]:
# 导入DEM、地质类型、河流和沟渠数据
dem = "DEM"
geology_type = "DZ"
rivers_shp = "river"
canals_shp = "vally"
# 填平DEM数据中的凹陷
filled_dem = "filled_dem"
arcpy.gp.Fill_sa(dem, filled_dem)


<geoprocessing server result object at 0x1aa5b6db1e0>

In [5]:
# 计算坡度
slope = "slope"
arcpy.gp.Slope_sa(filled_dem, slope, "DEGREE")

# 计算流向
flow_direction = "flow_direction"
arcpy.gp.FlowDirection_sa(filled_dem, flow_direction)

# 计算流积量
flow_accumulation = "flow_accumulation"
arcpy.gp.FlowAccumulation_sa(flow_direction, flow_accumulation)


<geoprocessing server result object at 0x1aa5b7182d0>

In [6]:
# 标准化DEM以确定相对位置（顶部、中部、下部）
# 计算局部最小值和最大值
neighborhood = NbrRectangle(101, 101, "CELL")
local_min = FocalStatistics(dem, neighborhood, "MINIMUM")
local_max = FocalStatistics(dem, neighborhood, "MAXIMUM")
mode_raster = local_max - local_min
calc_raster = dem - local_min
normalized_local = calc_raster / mode_raster

# 重新分类相对位置
remap = RemapRange([[0, 0.2, 1], [0.2, 0.7, 2], [0.7, 1, 3]])

# 使用 Reclassify 函数进行重新分类
out_reclassify = Reclassify(normalized_local, "VALUE", remap)
position = 'position'
out_reclassify.save(position)


In [7]:
# 设置像元大小为5M
arcpy.env.cellSize = 5  # 设置像元大小为5米
# 合并河流和沟渠图层
combined_features = "combined_features"
arcpy.Merge_management([rivers_shp, canals_shp], combined_features)

# 计算每个栅格单元到最近河流和沟渠的距离
distance_to_river_canal = "distance_to_river_canal"
arcpy.gp.EucDistance_sa(combined_features, distance_to_river_canal)


<geoprocessing server result object at 0x1aa5c1a8ab0>

In [3]:
# 标准化坡度
normalized_slope = "normalized_slope"
min_slope = arcpy.GetRasterProperties_management(slope, "MINIMUM").getOutput(0)
max_slope = arcpy.GetRasterProperties_management(slope, "MAXIMUM").getOutput(0)
expression = "((Raster('{}') - {}) / ({} - {}))".format(slope, min_slope, max_slope, min_slope)
arcpy.gp.RasterCalculator_sa(expression, normalized_slope)

# 标准化流积量
normalized_flow_accumulation = "normalized_flow_accumulation"
min_flow = arcpy.GetRasterProperties_management(flow_accumulation, "MINIMUM").getOutput(0)
max_flow = arcpy.GetRasterProperties_management(flow_accumulation, "MAXIMUM").getOutput(0)
expression = "((Raster('{}') - {}) / ({} - {}))".format(flow_accumulation, min_flow, max_flow, min_flow)
arcpy.gp.RasterCalculator_sa(expression, normalized_flow_accumulation)

# 标准化土壤类型
remap = RemapValue([
    [1, 0.1],  # 页岩
    [2, 0.5],  # 灰岩
    [3, 0.3],  # 第四纪红色粘土
    [4, 0.9],  # 砂岩
    [5, 0.6],  # 砂页岩
    [6, 0.8],  # 砾岩
    [7, 0.2],  # 水稻土
    [8, 0.85], # 第四纪冰川冲积物
    [9, 0.75]  # 紫红色砂岩
])

normalized_soil_type = Reclassify(geology_type, "VALUE", remap)
normalized_soil_type.save("normalized_soil_type")

# 标准化距离河流和沟渠的距离
normalized_distance = "normalized_distance"
min_distance = arcpy.GetRasterProperties_management(distance_to_river_canal, "MINIMUM").getOutput(0)
max_distance = arcpy.GetRasterProperties_management(distance_to_river_canal, "MAXIMUM").getOutput(0)
expression = "((Raster('{}') - {}) / ({} - {}))".format(distance_to_river_canal, min_distance, max_distance, min_distance)
arcpy.gp.RasterCalculator_sa(expression, normalized_distance)

# 标准化相对位置
normalized_position = "normalized_position"
expression = "((Raster('{}') - 1) / (3 - 1))".format(position)
arcpy.gp.RasterCalculator_sa(expression, normalized_position)


NameError: name 'slope' is not defined

In [ ]:
# 综合计算排水能力
drainage_capacity = "drainage_capacity"
expression = ("0.25 * Raster('{}') + 0.2 * Raster('{}') + 0.2 * Raster('{}') + "
              "0.2 * Raster('{}') + 0.1 * Raster('{}')").format(
    normalized_slope, normalized_flow_accumulation, normalized_soil_type,
    normalized_distance, normalized_position)
arcpy.gp.RasterCalculator_sa(expression, drainage_capacity)


In [ ]:
# 分类排水能力
classified_drainage_capacity = "classified_drainage_capacity.tif"
remap = RemapRange([[0, 0.25, 1], [0.25, 0.5, 2], [0.5, 0.75, 3], [0.75, 1, 4]])
arcpy.gp.Reclassify_sa(drainage_capacity, "VALUE", remap, classified_drainage_capacity)


In [ ]:
# 清理
arcpy.CheckInExtension("Spatial")
